# Imports

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

# Plot samples

In [ ]:
%run ../tools/augmentation.py
%run ../cxr14.py
%run ../iu_xray.py
%run ../jsrt.py
%run ../vinbig.py
%run ../chexpert.py

In [ ]:
dataset_kwargs = {
    'seg_multilabel': False,
    'masks': True,
    'masks_version': 'v2',
    'frontal_only': True,
    'image_size': (256, 256),
    'crop_center': 224,
}

# dataset = VinBigDataset(image_format='L', masks=True)
# dataset = CXR14Dataset(**dataset_kwargs)
dataset = ChexpertDataset(**dataset_kwargs)
# dataset = IUXRayDataset(**dataset_kwargs)
len(dataset)

In [ ]:
item = dataset[0]
item.image.size()

In [ ]:
item.masks.size()

In [ ]:
kwargs = {
    # 'mode': 'touch',
    'mode': 'single',
    'times': 1,
    'dont_shuffle': True,
    'seg_mask': dataset.enable_masks,
#     'crop': None,
    'translate': None,
#     'shear': None,
#     'rotation': None,
#     'contrast': None,
#     'brightness': None,
#     'noise_gaussian': None,
}

aug_dataset = Augmentator(dataset, **kwargs)
len(dataset), len(aug_dataset)

In [ ]:
aug_dataset.plot_augmented_samples(1, title_fontsize=28)

In [ ]:
sample_idx = 0
n_methods = len(aug_dataset._image_transforms)

start_idx = sample_idx * (n_methods + 1)
for i, method in enumerate(['original'] + list(aug_dataset._image_transforms)):
    idx = start_idx + i
    item = aug_dataset[idx]
    image = item.image
    print(f'{method:10}\t', image.type(), image.size(), image.min(), image.max())

    if aug_dataset.enable_masks:
        assert isinstance(item.masks, torch.Tensor), f'Masks not tensor, got {type(item.masks)}'
        mask = item.masks
        # print(f'{method:10}\t', mask.type(), mask.size(), mask.min(), mask.max())

# Gaussian noise

In [ ]:
%run ../../utils/common.py

In [ ]:
dataset2 = JSRTDataset(image_format='RGB')
len(dataset2)

In [ ]:
item = dataset2[3]
image = item.image
image.size()

In [ ]:
image.min(), image.max()

In [ ]:
image2 = image + torch.randn(image.size()[-2:], device=image.device) * 0.1
# image3 = tensor_to_range01(image) + torch.randn(image.size(), device=image.device) * 0.1

plt.figure(figsize=(15, 5))
n_rows = 2

plt.subplot(1, n_rows, 1)
plt.imshow(tensor_to_range01(image).permute(1, 2, 0), cmap='gray')

plt.subplot(1, n_rows, 2)
plt.imshow(tensor_to_range01(image2).permute(1, 2, 0), cmap='gray')

# plt.subplot(1, 3, 3)
# plt.imshow(image3, cmap='gray')

In [ ]:
noise = (image - image2)
noise.min(), noise.max(), noise.mean()